In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.linear_model import LogisticRegression, Ridge, Lasso, LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression
from ISLP import load_data
from ISLP.models import (ModelSpec as MS, Stepwise, sklearn_selected, sklearn_selection_path)
from functools import partial
from l0bnb import fit_path

c:\Users\admin\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\admin\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\admin\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
Hitters = load_data('Hitters')
np.isnan(Hitters['Salary']).sum()

59

In [5]:
Hitters = Hitters.dropna(how='any')
Hitters.shape

(263, 20)

In [6]:
def nCp(sigma2, estimator, X, y):
    n, p = X.shape
    y_hat = estimator.predict(X)
    RSS = np.sum((y - y_hat) ** 2)
    return -(RSS + 2*p*sigma2) / n

In [7]:
design = MS(Hitters.columns.drop('Salary')).fit(Hitters)
y = np.array(Hitters['Salary'])
X = design.transform(Hitters)
sigma2 = sm.OLS(y, X).fit().scale

In [8]:
neg_Cp = partial(nCp, sigma2)

In [9]:
strategy = Stepwise.first_peak(design, direction='forward', max_terms=len(design.terms))

In [11]:
hitters_Cp = sklearn_selected(sm.OLS, strategy, scoring=neg_Cp)
hitters_Cp.fit(Hitters, y)
hitters_Cp.selected_state_

('Assists',
 'AtBat',
 'CAtBat',
 'CRBI',
 'CRuns',
 'CWalks',
 'Division',
 'Hits',
 'PutOuts',
 'Walks')